script based on topohack notebook-- showing data comparison between IceSat2 ATL06 and a reference ground DEM (ASO) employing tools in the topolib library

In [10]:
# depedencies
import getpass
from topolib.subsetDat import subsetBBox;
from topolib import IceSat2Data;
import glob
from topolib import gda_lib;
from topolib import dwnldArctic
import numpy as np
import geopandas as gpd
from multiprocessing import Pool
import rasterio as rio
from rasterio.plot import plotting_extent
import rasterio.plot
import numpy as np
from pyproj import Proj, transform
from icepyx import icesat2data as ipd
# utility modules
import sys
import re
#import h5py
import matplotlib.pyplot as plt 
%matplotlib widget

adjust parameters and bounds

In [2]:
# inputs
# if want to explcitly define bounding box, raster = None
datafolder = '/home/jovyan/shared/data-aragon/'
datafolderOut = '/home/jovyan/shared/data-aragon/ATL06/CA_atl06/' #

#if have raster define raster, if not, use None
raster = datafolder+'ASO_3M_PCDTM_USCATE_20191010_20191010.tif'
#raster = None
# if defining extent using the raster, then rasterBnd = 1, else rasterBnd = 2
rasterBnd = 1

if rasterBnd == None:
    # Area of interest
    boundaries = {
        'LowerLeft_Lon': '-114.4', #-120.6',
        'LowerLeft_Lat': '62.4', #37.7',
        'UpperRight_Lon': '-114.3',#-119.0',
        'UpperRight_Lat': '62.5',#38.94',
    }
#'bounding_box': '-114.4,62.446,-114.38,62.47'spatial_extent = [L,B,R,T]
# Specify the variables of interest
LIce_var = ['atl06_quality_summary','delta_time','h_li','hli_sigma',\
           'latitude','longitude','segment_id','sigma_geo_h']
variables = {
    'beams': [
        '/land_ice_segments/'+LIce_var[0],
        '/land_ice_segments/'+LIce_var[1],
        '/land_ice_segments/'+LIce_var[2],
        '/land_ice_segments/'+LIce_var[3],
        '/land_ice_segments/'+LIce_var[4],
        '/land_ice_segments/'+LIce_var[5],
        '/land_ice_segments/'+LIce_var[6],
        '/land_ice_segments/'+LIce_var[7],
        '/ancillary_data/atlas_sdp_gps_epoch',
    ],
    'other': [
        '/orbit_info/cycle_number',
        '/orbit_info/rgt',
        '/orbit_info/orbit_number',
    ]
}

# Custom time range of interest
# define full_per = 1
full_per = 1
time_range = {
        'start_date': '2019-01-20',
        'end_date': '2019-04-30'
}

# Enter your Earthdata Login user name
user_id = 'jawairiaahmad'
# Enter your email address associated with your Earthdata Login account
email = 'jahmad@umd.edu'

In [3]:
def subsetBBox(rast,proj_in,proj_out):

    # rasterio open data
    rB = rio.open(rast)
    # rasterio get bounding box
    [L,B,R,T] = rB.bounds

    if proj_in == proj_out:
        return L, R, T, B
    else:
        incord = Proj(init=proj_in)
        outcord = Proj(init=proj_out)

        [Left,Bottom] = transform(incord,outcord,L,B)
        [Right,Top] = transform(incord,outcord,R,T)
        return Left, Bottom, Right, Top

In [4]:
import os
import shutil
from pprint import pprint
%matplotlib inline
#%load_ext autoreload
#%autoreload 2

In [5]:
#path to raster DEM
raster_path = raster#'/home/jovyan/shared/data-aragon/ASO_3M_PCDTM_USCATE_20191010_20191010.tif'
ds = rasterio.open(raster_path)
dem = ds.read(1)
xmin,ymin,xmax,ymax = ds.bounds
ds.crs
raster = rio.open(raster_path)
raster.meta

{'driver': 'GTiff',
 'dtype': 'float32',
 'nodata': None,
 'width': 21623,
 'height': 18653,
 'count': 1,
 'crs': CRS.from_epsg(32611),
 'transform': Affine(3.0, 0.0, 242715.0,
        0.0, -3.0, 4234848.0)}

In [6]:
[L,B,R,T] = subsetBBox(raster_path,raster.crs,'epsg:4326') # now read projection from DEM, convert to WGS84
    #[L,B,R,T] = subsetBBox(raster,'epsg:32611','epsg:4326')
    # Area of interest
boundaries = {
    'LowerLeft_Lon': L,
    'LowerLeft_Lat': B,
    'UpperRight_Lon': R,
    'UpperRight_Lat': T,
}
L,B,R,T

/srv/conda/envs/notebook/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/srv/conda/envs/notebook/lib/python3.7/site-packages/pyproj/crs/crs.py:294: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))
/srv/conda/envs/notebook/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.

(-119.91910216526328,
 37.721153213268956,
 -119.19868268078534,
 38.24106453792272)

In [17]:
# Earthdata Login credentials
password = getpass.getpass('Earthdata Login password: ')

# Setup a search session
search = IceSat2Data(user_id, password, variables=variables)

Earthdata Login password:  ··········


In [60]:
# Search for granules in bounding box without download through all datasets
search.search_granules(bounding_box=boundaries)

# Search for granules in bounding box without download for specific time range
search.search_granules(bounding_box=boundaries, time_range=time_range)

Number of granules:
    40
Average size of granules in MB:
    45.2853816747625
Total size in MB:
    1811.4152669905002
Number of granules:
    8
Average size of granules in MB:
    61.3552069664
Total size in MB:
    490.84165573120004


8

In [61]:
# NOTE: file paths have to be explicitly defined 
# BEFORE submitting 

params = {}
if full_per:
    params['time_range'] = time_range

# Submit a download order to NSIDC with no time restriction
search.order_data(
    email,
    datafolderOut,
    boundaries,
    **params
)

Number of granules:
    8
Average size of granules in MB:
    61.3552069664
Total size in MB:
    490.84165573120004
Order:  1
Request HTTP response:  201
order ID:  5000000701984
Data request  1  is submitting...
Initial request status is  processing
Status is not complete. Trying again.
Retry request status is:  complete_with_errors
error messages:
['177534408:NoMatchingData - No data found that matched subset constraints. '
 'Exit code 3.',
 '178003329:NoMatchingData - No data found that matched subset constraints. '
 'Exit code 3.',
 '177973035:NoMatchingData - No data found that matched subset constraints. '
 'Exit code 3.',
 '178001530:NoMatchingData - No data found that matched subset constraints. '
 'Exit code 3.',
 '178019838:NoMatchingData - No data found that matched subset constraints. '
 'Exit code 3.',
 'PT2.570S',
 'ICESAT2']
Beginning download of zipped output...
Data request 1 is complete.


OSError: [Errno 39] Directory not empty: '/home/jovyan/SeasonalSnow/contributors/Jawairia/.ipynb_checkpoints'

In [7]:
# glob to list of files
ATL06_list = glob.glob(datafolderOut+'*.h5')

In [8]:
# del ATL06_list[2] # x.to_crs crashed on this file
ATL06_list

['/home/jovyan/SeasonalSnow/contributors/Jawairia/processed_ATL06_20191008182255_01810506_003_01.h5']

In [5]:
# read hdf5 into a geodataframe
dataset_dict={'land_ice_segments':LIce_var, 'land_ice_segments/ground_track':['x_atc']}


In [79]:
## use this box if looking at one .h5 file -- one track at a certain time
ATL06_gdf_list = [gda_lib.ATL06_2_gdf(ATL06_list[0],dataset_dict)]
ATL06_gdf_list = [ATL06_gdf_list[0].to_crs(ds.crs)]
big_gdf = gda_lib.concat_gdf(ATL06_gdf_list)
#ATL06_gdf_list[0]

/home/jovyan/topohack/topolib/gda_lib.py:33: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  with h5py.File(filename) as h5f:
/srv/conda/envs/notebook/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [9]:
## use this box if looking at more than one .h5 file -- multiple tracks or same track at different times
#ATL06_gdf_list = [gda_lib.ATL06_2_gdf(x,dataset_dict) for x in ATL06_list]
#ATL06_gdf_list = [x.to_crs(ds.crs) for x in ATL06_gdf_list]
#ATL06_gdf_list;
#big_gdf = gda_lib.concat_gdf(ATL06_gdf_list)

/home/jovyan/topohack/topolib/gda_lib.py:33: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  with h5py.File(filename) as h5f:
/srv/conda/envs/notebook/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [11]:
#plot DEM and overlapping ATL06 lines
%matplotlib inline

fig,ax = plt.subplots(figsize=(10,8))
im = ax.imshow(dem,cmap='gray',extent=[xmin,xmax,ymin,ymax])
#ATL06_gdf_list[0].plot(ax=ax)
for gdf in ATL06_gdf_list:
    gdf.plot(ax=ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
# separate flight tracks for line-by-line comparison
#df1,df2,df3,df4,df5,df6 = [x for x_,x in ATL06_gdf_list[1].groupby(ATL06_gdf_list[1]['p_b'])] #pick 1 ATL06 and seperate it into 6
df1,df2,df3,df4,df5,df6 = [x for x_,x in ATL06_gdf_list[0].groupby(ATL06_gdf_list[0]['p_b'])] #pick 1 ATL06 and seperate it into 6
df_list = [df3]#,df3,df5]#,df2,df3,df4,df5,df6]
#df_list


In [13]:
# comparison of ground data and ATL06 using a 20m buffer sampler
# colors represent the flight lines shown above
df_test_big = gda_lib.buffer_sampler(ds,big_gdf,20,ret_gdf=True)
df_test_big['snow_on_off'] = df_test_big['h_li']-df_test_big['med']

fig,ax = plt.subplots()
for gdf in df_list:
    df_test = gda_lib.buffer_sampler(ds,gdf,20,ret_gdf=True)
    df_test['snow_on_off'] = df_test['h_li']-df_test['med']
    ax.scatter(df_test['latitude'],df_test['snow_on_off'],label='Beam:'+df_test['p_b'][0])
    ax.set_ylabel('Diff. b/t h_li and DEM')
    ax.set_xlabel('latitude')
    plt.legend()
    print(df_test['snow_on_off'].mean(),df_test['snow_on_off'].std(),100*df_test['snow_on_off'].mean()/(df_test['h_li'].max()-df_test['h_li'].min()))
# df_test['snow_on_off'].hist(bins=30)

big_gdf['diff'] = df_test_big['snow_on_off']

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.2736438998035678 2.8596413610845173 0.015976457824357932


In [14]:
# point-by-point comparison
fig,ax = plt.subplots(figsize=(10,8))
im = ax.imshow(dem,cmap='inferno',extent=[xmin,xmax,ymin,ymax])
big_gdf.plot('h_li',legend=True,ax=ax,cmap='inferno',s=1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
## comparison of ATL06 data over the same track from two different time periods
datafolderOut2 = '/home/jovyan/shared/data-aragon/ATL06/canada_lake_domain/'
ATL06_list2 = glob.glob(datafolderOut2+'*.h5')
ATL06_list2

['/home/jovyan/shared/data-aragon/ATL06/canada_lake_domain/processed_ATL06_20190128175143_04780203_003_01.h5',
 '/home/jovyan/shared/data-aragon/ATL06/canada_lake_domain/processed_ATL06_20190429133124_04780303_003_01.h5']

In [21]:
## use this box if looking at more than one .h5 file -- multiple tracks or same track at different times
ATL06_gdf_list2 = [gda_lib.ATL06_2_gdf(x,dataset_dict) for x in ATL06_list2]
#ATL06_gdf_list2 = [x.to_crs(ds.crs) for x in ATL06_gdf_list2]
big_gdf2 = gda_lib.concat_gdf(ATL06_gdf_list2)
#ATL06_gdf_list2[1]

/srv/conda/envs/notebook/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [22]:
df1_a,df2_a = [x for x_,x in ATL06_gdf_list2[0].groupby(ATL06_gdf_list2[0]['p_b'])] #pick 1 ATL06 and seperate it into 6
df1_b,df2_b = [x for x_,x in ATL06_gdf_list2[1].groupby(ATL06_gdf_list2[1]['p_b'])] #pick 1 ATL06 and seperate it into 6
#df_list2 = [df1_b,df2_b,df3_b,df4_b,df5_b,df6_b] ,,df3_b,df4_b,df5_b,df6_b
#df_list2;

In [23]:
fig,ax = plt.subplots(figsize=(6,5))
ax.scatter(df1_a['latitude'],df1_a['h_li'],c='blue',label='Jan-Beam:'+df1_a['p_b'][0])
ax.scatter(df1_b['latitude'],df1_b['h_li'],c='green',label='Apr-Beam:'+df1_b['p_b'][1])
ax.scatter(df2_a['latitude'],df2_a['h_li'],c='red',label='Jan-Beam:'+df2_a['p_b'][0])
ax.scatter(df2_b['latitude'],df2_b['h_li'],c='cyan',label='Apr-Beam:'+df2_b['p_b'][1])
ax.set_ylabel('h_li')
ax.set_xlabel('latitude')
plt.legend()
print(df1_a['delta_time'][0],df1_b['delta_time'][0])


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

33933149.01369724 41779929.63442679
